In [37]:
import torch
from torch_geometric.nn import MessagePassing
import torch_geometric.nn
from torch_geometric.nn import MLP
from torch_geometric.data import Data
print('pytorch version = ',torch.__version__)
gpu_id = '0'
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('GPU name: {:s}, gpu_id: {:s}'.format(torch.cuda.get_device_name(0),gpu_id))  

pos = torch.tensor([[2,3],[4,5],[6,5]], dtype=torch.long)
edge_index = torch.tensor([[100, 100],
                           [150, 870],
                           [130, 200],], dtype=torch.long)
x = torch.tensor([[-1,4], [0,3], [1,6], [3,4], [2,44]], dtype=torch.float)

data = Data(pos=pos, x=x, edge_index=edge_index.t().contiguous())
mlp = MLP(in_channels=16, hidden_channels=32,
          out_channels=128, num_layers=3)
print(data.keys)
print(data.has_self_loops())
print(data.has_isolated_nodes())
print(data.num_features)
print(data.num_nodes)
print(data.num_edges)

pytorch version =  1.10.0
GPU name: NVIDIA GeForce RTX 2080 Ti, gpu_id: 0
['x', 'pos', 'edge_index']
True
True
2
5
3


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from torch_geometric.utils.convert import to_networkx, from_networkx
import torch


# Make the networkx graph
G = nx.Graph()

# Add some cars (just do 4 for now)
G.add_nodes_from([
      (683,152,0,3),
      (623,132,0,3),
      (456,142,0,1),
      (432,192,0,2),
])

# Add some edges
G.add_edges_from([
                  (1, 2), (1, 4), (1, 5),
                  (2, 3), (2, 4),
                  (3, 2), (3, 5), 
                  (4, 1), (4, 2),
                  (5, 1), (5, 3)
])

# Convert the graph into PyTorch geometric
pyg_graph = from_networkx(G)

In [ ]:
embed_dim = 128
from torch_geometric.nn import TopKPooling
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = SAGEConv(embed_dim, 128)
        self.pool1 = TopKPooling(128, ratio=0.8)
        self.conv2 = SAGEConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.8)
        self.conv3 = SAGEConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.8)
        self.item_embedding = torch.nn.Embedding(num_embeddings=df.item_id.max() +1, embedding_dim=embed_dim)
        self.lin1 = torch.nn.Linear(256, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, 1)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()        
  
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.item_embedding(x)
        x = x.squeeze(1)        

        x = F.relu(self.conv1(x, edge_index))

        x, edge_index, _, batch, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
     
        x, edge_index, _, batch, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)      
        x = F.dropout(x, p=0.5, training=self.training)

        x = torch.sigmoid(self.lin3(x)).squeeze(1)

        return x

In [ ]:
def train():
    model.train()

    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)
    
device = torch.device('cuda')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
crit = torch.nn.BCELoss()
train_loader = DataLoader(train_dataset, batch_size=batch_size)
for epoch in range(num_epochs):
    train()

In [90]:
import torch
from torch_geometric.nn import knn_graph
from torch_geometric.loader import DataLoader

x = torch.Tensor([[-1, -1], [-1, 1], [1, -1], [1, 1]])
dataset = x.repeat(128,1)
x = x.repeat(128,1,1)
print(x)
loader = DataLoader(dataset, batch_size=128, shuffle=True)
edge_index = torch.tensor([])
for x_seed in x:
    edge_index_temp = knn_graph(x_seed, k=1, batch=batch, loop=False)
    edge_index_temp = torch.unsqueeze(edge_index_temp, 0)
    edge_index = torch.cat((edge_index, edge_index_temp),0)
edge_index = torch.Tensor(edge_index).type(torch.long)
print(edge_index.shape)

tensor([[[-1., -1.],
         [-1.,  1.],
         [ 1., -1.],
         [ 1.,  1.]],

        [[-1., -1.],
         [-1.,  1.],
         [ 1., -1.],
         [ 1.,  1.]],

        [[-1., -1.],
         [-1.,  1.],
         [ 1., -1.],
         [ 1.,  1.]],

        ...,

        [[-1., -1.],
         [-1.,  1.],
         [ 1., -1.],
         [ 1.,  1.]],

        [[-1., -1.],
         [-1.,  1.],
         [ 1., -1.],
         [ 1.,  1.]],

        [[-1., -1.],
         [-1.,  1.],
         [ 1., -1.],
         [ 1.,  1.]]])
torch.Size([128, 2, 4])


In [85]:
import torch
from torch_geometric.nn import knn_graph

x = torch.Tensor([[-1, -1], [-1, 1], [1, -1], [1, 1]])
batch = torch.tensor([0, 0, 0, 0])
edge_index = knn_graph(x, k=2, batch=batch, loop=False)
print(edge_index)

tensor([[1, 2, 0, 3, 0, 3, 1, 2],
        [0, 0, 1, 1, 2, 2, 3, 3]])
